In [ ]:
# ====================================================================
# PROJET BUSINESS INTELLIGENCE - TECHSTORE
# ETL PIPELINE - PARTIE MEMBRE 1 : DATA EXTRACTION
# ====================================================================

# %% [markdown]
# # 📊 Extraction des Données (Membre 1)
# 
# Ce notebook contient la partie extraction du pipeline ETL.
# **Responsable :** Membre 1 - Data Extraction Engineer
# 
# ## Objectifs :
# 1. ✅ Extraire les données de MySQL (ERP)
# 2. ✅ Scraper les prix des concurrents
# # 3. ✅ Valider la qualité des données extraites

# %% Imports pour tout le pipeline ETL
# Installer les dépendances manquantes (exécuter uniquement dans le notebook)


# === Imports de base ===
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

# === Partie 1 : Extraction (Membre 1) ===
import mysql.connector
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# === Partie 2 : Transformations (Membre 2 - Toi) ===
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pytesseract
from PIL import Image
import re
from fuzzywuzzy import process  # Pour le matching des prix concurrents

print("✅ Bibliothèques importées avec succès")
print(f"📅 Date d'exécution: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# %% [markdown]
# ## 1️⃣ Configuration de la Connexion MySQL

# %% Configuration
MYSQL_CONFIG = {
    'host': 'boughida.com',
    'database': 'techstore_erp',
    'user': 'student_user_4ing',
    'password': 'bi_guelma_2025'
}

# Créer les répertoires nécessaires
os.makedirs('data/extracted', exist_ok=True)

print("📋 Configuration chargée")
print(f"   Serveur: {MYSQL_CONFIG['host']}")
print(f"   Base de données: {MYSQL_CONFIG['database']}")

# %% [markdown]
# ## 2️⃣ Connexion et Test

# %% Test de connexion
def test_connection():
    """Tester la connexion à MySQL"""
    try:
        conn = mysql.connector.connect(**MYSQL_CONFIG)
        if conn.is_connected():
            print("✅ Connexion MySQL réussie!")
            
            # Tester une requête simple
            cursor = conn.cursor()
            cursor.execute("SELECT DATABASE()")
            db_name = cursor.fetchone()[0]
            print(f"   Base de données active: {db_name}")
            
            # Lister les tables disponibles
            cursor.execute("SHOW TABLES")
            tables = cursor.fetchall()
            print(f"   Nombre de tables: {len(tables)}")
            
            conn.close()
            return True
    except Exception as e:
        print(f"❌ Erreur de connexion: {e}")
        return False

# Exécuter le test
test_connection()

# %% [markdown]
# ## 3️⃣ Extraction des Tables MySQL

# %% Fonction d'extraction
def extract_mysql_table(table_name, conn):
    """Extraire une table MySQL vers DataFrame"""
    try:
        print(f"📊 Extraction: {table_name}...", end=" ")
        
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, conn)
        
        # Sauvegarder en CSV
        filename = f"data/extracted/{table_name.replace('table_', '')}.csv"
        df.to_csv(filename, index=False, encoding='utf-8')
        
        print(f"✅ {len(df)} lignes | {len(df.columns)} colonnes")
        
        return df
        
    except Exception as e:
        print(f"❌ Erreur: {e}")
        return None

# %% Extraction de toutes les tables
print("\n" + "="*60)
print("🚀 EXTRACTION DES TABLES MySQL")
print("="*60 + "\n")

# Se connecter
conn = mysql.connector.connect(**MYSQL_CONFIG)

# Liste des tables à extraire
tables_to_extract = [
    'table_sales',
    'table_products',
    'table_reviews',
    'table_customers',
    'table_stores',
    'table_cities',
    'table_categories',
    'table_subcategories'
]

# Dictionnaire pour stocker les DataFrames
dataframes = {}

# Extraire chaque table
for table in tables_to_extract:
    df = extract_mysql_table(table, conn)
    if df is not None:
        # Enlever le préfixe "table_" pour le nom
        clean_name = table.replace('table_', '')
        dataframes[clean_name] = df

# Fermer la connexion
conn.close()

print("\n✅ Extraction MySQL terminée!")
print(f"📦 {len(dataframes)} tables extraites")

# %% [markdown]
# ## 4️⃣ Aperçu des Données Extraites

# %% Afficher un aperçu
print("\n" + "="*60)
print("📋 APERÇU DES DONNÉES EXTRAITES")
print("="*60 + "\n")

for name, df in dataframes.items():
    print(f"\n📊 Table: {name}")
    print(f"   Dimensions: {df.shape[0]} lignes × {df.shape[1]} colonnes")
    print(f"   Colonnes: {', '.join(df.columns.tolist()[:5])}...")
    print(f"   Aperçu:")
    display(df.head(3))
    print("-" * 60)

# %% [markdown]
# ## 5️⃣ Statistiques des Ventes

# %% Analyse rapide des ventes
df_sales = dataframes['sales']

print("\n📊 STATISTIQUES DES VENTES")
print("="*60)
print(f"Nombre total de ventes: {len(df_sales):,}")
print(f"Revenu total: {df_sales['Total_Revenue'].sum():,.2f} DZD")
print(f"Revenu moyen par vente: {df_sales['Total_Revenue'].mean():,.2f} DZD")
print(f"Période: {df_sales['Date'].min()} → {df_sales['Date'].max()}")

# %% [markdown]
# ## 6️⃣ Web Scraping - Prix Concurrents

# %% Configuration du scraping
COMPETITOR_URL = "https://boughida.com/competitor/"

print("\n" + "="*60)
print("🕷️  WEB SCRAPING - PRIX CONCURRENTS")
print("="*60 + "\n")

# %% Fonction de scraping
def scrape_competitor_prices(url):
    """Scraper les prix des concurrents"""
    try:
        print(f"📡 Connexion à: {url}")
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        products = []
        
        # Chercher les produits (adapter selon la structure HTML)
        product_items = soup.find_all('div', class_='product-item')
        
        print(f"🔍 {len(product_items)} produits trouvés")
        
        for item in product_items:
            try:
                # Extraire le nom
                name_elem = item.find('h3')
                name = name_elem.text.strip() if name_elem else None
                
                # Extraire le prix
                price_elem = item.find('span', class_='price')
                if price_elem:
                    price_text = price_elem.text.strip()
                    # Nettoyer le prix
                    import re
                    price = float(re.sub(r'[^\\d.]', '', price_text))
                else:
                    price = None
                
                if name and price:
                    products.append({
                        'Competitor_Product_Name': name,
                        'Competitor_Price': price
                    })
                    
            except Exception as e:
                continue
        
        return pd.DataFrame(products)
        
    except Exception as e:
        print(f"❌ Erreur de scraping: {e}")
        print("📝 Utilisation de données de test à la place")
        return create_test_competitor_data()

# %% Fonction de données de test
def create_test_competitor_data():
    """Créer des données de test pour les prix concurrents"""
    test_data = [
        {'Competitor_Product_Name': 'Laptop HP ProBook 450', 'Competitor_Price': 95000},
        {'Competitor_Product_Name': 'Dell Latitude 5420', 'Competitor_Price': 105000},
        {'Competitor_Product_Name': 'iPhone 14 128GB', 'Competitor_Price': 165000},
        {'Competitor_Product_Name': 'Samsung S23 Ultra', 'Competitor_Price': 185000},
        {'Competitor_Product_Name': 'Sony WH-1000XM5', 'Competitor_Price': 42000},
        {'Competitor_Product_Name': 'AirPods Pro 2', 'Competitor_Price': 35000},
        {'Competitor_Product_Name': 'LG OLED55C3', 'Competitor_Price': 215000},
        {'Competitor_Product_Name': 'Samsung QN65Q80C', 'Competitor_Price': 275000},
    ]
    return pd.DataFrame(test_data)

# %% Exécuter le scraping
df_competitor = scrape_competitor_prices(COMPETITOR_URL)

# Sauvegarder
df_competitor.to_csv('data/extracted/competitor_prices.csv', index=False)

print(f"\n✅ Scraping terminé: {len(df_competitor)} produits extraits")
print(f"💾 Fichier sauvegardé: data/extracted/competitor_prices.csv")

# Afficher un aperçu
print("\n📋 Aperçu des prix concurrents:")
display(df_competitor.head(10))

# %% [markdown]
# ## 7️⃣ Validation des Données Extraites

# %% Vérifications de qualité
print("\n" + "="*60)
print("✅ VALIDATION DES DONNÉES")
print("="*60 + "\n")

validation_results = []

for name, df in dataframes.items():
    # Vérifier les valeurs manquantes
    missing = df.isnull().sum().sum()
    missing_pct = (missing / (df.shape[0] * df.shape[1])) * 100
    
    # Vérifier les doublons
    duplicates = df.duplicated().sum()
    
    validation_results.append({
        'Table': name,
        'Lignes': len(df),
        'Colonnes': len(df.columns),
        'Valeurs_Manquantes': missing,
        'Pct_Manquant': f"{missing_pct:.2f}%",
        'Doublons': duplicates,
        'Statut': '✅' if missing_pct < 5 and duplicates < 10 else '⚠️'
    })

df_validation = pd.DataFrame(validation_results)
display(df_validation)

# %% [markdown]
# ## 8️⃣ Résumé de l'Extraction

# %% Résumé final
print("\n" + "="*70)
print("📊 RÉSUMÉ DE L'EXTRACTION (MEMBRE 1)")
print("="*70 + "\n")

print("✅ TÂCHES COMPLÉTÉES:")
print("   1. Connexion MySQL établie et testée")
print("   2. 8 tables extraites de l'ERP")
print("   3. Web scraping des prix concurrents effectué")
print("   4. Toutes les données sauvegardées en CSV")
print("   5. Validation de la qualité des données effectuée")

print("\n📦 FICHIERS CRÉÉS:")
for file in os.listdir('data/extracted'):
    file_path = f"data/extracted/{file}"
    file_size = os.path.getsize(file_path) / 1024  # KB
    print(f"   • {file} ({file_size:.1f} KB)")

print("\n🚀 PROCHAINE ÉTAPE:")
print("   → Membre 2 peut maintenant transformer ces données")
print("   → Fichiers disponibles dans: data/extracted/")

print("\n" + "="*70)
print("✅ EXTRACTION TERMINÉE AVEC SUCCÈS")
print("="*70)





# %% [markdown]
# ## PARTIE 2 : Transformations des Données (Hadjer)
#
# **Responsable** : Hadjer – Data Cleaning & Feature Engineering Specialist

# %% Configuration initiale
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import os

# Créer le dossier de sortie
os.makedirs('data/transformed', exist_ok=True)

# %% Chargement des données
print("🚀 Chargement des données extraites...\n")

sales_df        = pd.read_csv('data/extracted/sales.csv')
products_df     = pd.read_csv('data/extracted/products.csv')
customers_df    = pd.read_csv('data/extracted/customers.csv')
cities_df       = pd.read_csv('data/extracted/cities.csv')
categories_df   = pd.read_csv('data/extracted/categories.csv')
reviews_df      = pd.read_csv('data/extracted/reviews.csv') if 'reviews.csv' in os.listdir('data/extracted') else None
stores_df       = pd.read_csv('data/extracted/stores.csv') if 'stores.csv' in os.listdir('data/extracted') else None

# Chargement optionnel
try:
    subcategories_df = pd.read_csv('data/extracted/subcategories.csv')
    print("✓ subcategories.csv chargé")
except FileNotFoundError:
    subcategories_df = None
    print("⚠ subcategories.csv non trouvé")

print(f"\nDonnées chargées : {len(sales_df)} ventes, {len(products_df)} produits")
print("Colonnes dans sales.csv :", sales_df.columns.tolist())

# %% 1. Aperçu initial et valeurs manquantes
print("\n" + "="*70)
print("1. Aperçu initial et valeurs manquantes")
print("="*70)

display(sales_df.head(5))
print("\nValeurs manquantes par colonne :")
print(sales_df.isnull().sum())

# %% 2. Détection et normalisation des colonnes critiques
print("\n" + "="*70)
print("2. Détection automatique des colonnes essentielles")
print("="*70)

# --- Colonne date ---
possible_dates = ['Date', 'date', 'order_date', 'sale_date', 'Order_Date']
date_col = next((col for col in possible_dates if col in sales_df.columns), None)
if date_col:
    sales_df = sales_df.rename(columns={date_col: 'date'})
    sales_df['date'] = pd.to_datetime(sales_df['date'], errors='coerce')
    sales_df['year'] = sales_df['date'].dt.year
    sales_df['month'] = sales_df['date'].dt.month
    print(f"✓ Colonne date détectée et normalisée : '{date_col}' → 'date'")
else:
    print("⚠ Aucune colonne date trouvée → création de placeholders")
    sales_df['date'] = pd.NaT
    sales_df['year'] = pd.NA
    sales_df['month'] = pd.NA

# --- Colonne order_value_EUR (valeur de vente) ---
possible_values = ['order_value_EUR', 'Order_Value_EUR', 'Total_Revenue', 'total_revenue', 'revenue', 'price']
value_col = next((col for col in possible_values if col in sales_df.columns), None)
if value_col:
    sales_df = sales_df.rename(columns={value_col: 'order_value_EUR'})
    sales_df['order_value_EUR'] = pd.to_numeric(sales_df['order_value_EUR'], errors='coerce')
    print(f"✓ Colonne valeur vente détectée : '{value_col}' → 'order_value_EUR'")
else:
    raise KeyError("Aucune colonne de valeur de vente trouvée !")

# --- Colonne cost (coût) ---
possible_costs = ['cost', 'Cost', 'unit_cost', 'Unit_Cost', 'cost_price', 'Cost_Price', 'total_cost', 'Total_Cost']
cost_col = next((col for col in possible_costs if col in sales_df.columns), None)
if cost_col:
    sales_df = sales_df.rename(columns={cost_col: 'cost'})
    sales_df['cost'] = pd.to_numeric(sales_df['cost'], errors='coerce')
    print(f"✓ Colonne coût détectée : '{cost_col}' → 'cost'")
else:
    print("⚠ Aucune colonne coût trouvée → création avec NaN")
    sales_df['cost'] = np.nan

# --- Autres colonnes utiles (optionnelles) ---
for std, possibles in {'device_type': ['device_type', 'Device_Type', 'channel'],
                       'category': ['category', 'Category', 'product_category'],
                       'country': ['country', 'Country'],
                       'city': ['city', 'City']}.items():
    col = next((c for c in possibles if c in sales_df.columns), None)
    if col and col != std:
        sales_df = sales_df.rename(columns={col: std})

# %% 3. Nettoyage des valeurs non numériques (cost et order_value_EUR)
print("\n" + "="*70)
print("3. Nettoyage des colonnes numériques")
print("="*70)

for col in ['order_value_EUR', 'cost']:
    if col in sales_df.columns:
        invalid = sales_df[col].isna()
        print(f"{col} - Lignes invalides/après coerce : {invalid.sum()}")
        if invalid.sum() > 0:
            display(sales_df[invalid].head(3))

# Supprimer lignes où order_value_EUR ou cost sont NaN (critiques)
sales_df.dropna(subset=['order_value_EUR'], inplace=True)
if 'cost' in sales_df.columns:
    sales_df.dropna(subset=['cost'], inplace=True)

print(f"✓ Nettoyage terminé → {len(sales_df)} lignes restantes")

# %% 4. Imputation des valeurs manquantes restantes
print("\n" + "="*70)
print("4. Imputation")
print("="*70)

sales_df['order_value_EUR'].fillna(sales_df['order_value_EUR'].mean(), inplace=True)
if 'cost' in sales_df.columns:
    sales_df['cost'].fillna(sales_df['cost'].mean(), inplace=True)

if 'device_type' in sales_df.columns:
    mode_device = sales_df['device_type'].mode()[0] if not sales_df['device_type'].mode().empty else 'unknown'
    sales_df['device_type'].fillna(mode_device, inplace=True)
    print(f"device_type imputé avec : '{mode_device}'")

# %% 5. Suppression des doublons
print("\n" + "="*70)
print("5. Suppression des doublons")
print("="*70)

duplicates = sales_df.duplicated().sum()
sales_df.drop_duplicates(inplace=True)
print(f"✓ {duplicates} doublons supprimés → {len(sales_df)} lignes")

# %% 6. Création de profit_margin
print("\n" + "="*70)
print("6. Création profit_margin")
print("="*70)

if 'cost' in sales_df.columns:
    sales_df['profit_margin'] = sales_df['order_value_EUR'] - sales_df['cost']
else:
    sales_df['profit_margin'] = np.nan
    print("⚠ profit_margin mis à NaN (pas de colonne cost)")

display(sales_df[['order_value_EUR', 'cost', 'profit_margin']].head())

# %% 7. Analyse des profits négatifs
print("\n" + "="*70)
print("7. Transactions en perte")
print("="*70)

if 'profit_margin' in sales_df.columns:
    negative = sales_df[sales_df['profit_margin'] < 0]
    print(f"Nombre de ventes en perte : {len(negative)}")
    if len(negative) > 0:
        display(negative.head(10))

# %% 8. Standardisation texte
print("\n" + "="*70)
print("8. Standardisation colonnes texte")
print("="*70)

text_cols = ['country', 'category', 'city', 'device_type']
for col in text_cols:
    if col in sales_df.columns:
        sales_df[col] = sales_df[col].astype(str).str.strip().str.lower().str.replace('-', ' ')

# %% 9. Colonnes dérivées
print("\n" + "="*70)
print("9. Colonnes dérivées + top catégories")
print("="*70)

if 'profit_margin' in sales_df.columns and (sales_df['order_value_EUR'] != 0).any():
    sales_df['margin_rate'] = sales_df['profit_margin'] / sales_df['order_value_EUR']
    sales_df['margin_rate'] = sales_df['margin_rate'].round(4)
    sales_df['profit_per_1000EUR'] = (sales_df['profit_margin'] / 1000).round(2)

    if 'category' in sales_df.columns:
        top_cat = sales_df.groupby('category')['margin_rate'].mean().sort_values(ascending=False).head(5) * 100
        print("Top 5 catégories les plus rentables (%) :")
        display(top_cat)

# %% 10. Nettoyage incohérences business
print("\n" + "="*70)
print("10. Suppression incohérences")
print("="*70)

bad = (sales_df['order_value_EUR'] <= 0) | ((sales_df['order_value_EUR'] < sales_df['cost']) if 'cost' in sales_df.columns else False)
bad_count = bad.sum()
sales_df = sales_df[~bad].copy()
print(f"✓ {bad_count} lignes incohérentes supprimées → {len(sales_df)} lignes valides")

# Sauvegarde cleaned
sales_df.to_csv('data/transformed/sales_cleaned.csv', index=False)
sales_df.to_excel('data/transformed/sales_cleaned.xlsx', index=False)
print("💾 Fichiers cleaned sauvegardés dans data/transformed/")

# %% 11. Ajout 3 lignes NaN + backward fill
print("\n" + "="*70)
print("11. Ajout 3 lignes NaN + backward fill")
print("="*70)

empty = pd.DataFrame(index=range(3), columns=sales_df.columns)
sales_df = pd.concat([empty, sales_df], ignore_index=True)
sales_df = sales_df.bfill()

print("✓ 3 lignes ajoutées et backward fill appliqué")

# %% 12. Scaling
print("\n" + "="*70)
print("12. Scaling (MinMax & Standard)")
print("="*70)

num_cols = [col for col in ['order_value_EUR', 'cost', 'profit_margin'] if col in sales_df.columns]

if num_cols:
    minmax = MinMaxScaler().fit_transform(sales_df[num_cols])
    std = StandardScaler().fit_transform(sales_df[num_cols])

    comparison = pd.DataFrame({
        **{f"{c}_orig": sales_df[c].head(10).round(2) for c in num_cols},
        **{f"{c}_minmax": minmax[:10, i].round(4) for i, c in enumerate(num_cols)},
        **{f"{c}_std": std[:10, i].round(4) for i, c in enumerate(num_cols)},
    })
    display(comparison)
else:
    print("⚠ Aucune colonne numérique pour scaling")

print("\n" + "="*70)
print("🎉 PARTIE 2 TERMINÉE AVEC SUCCÈS !")
print("="*70)

✅ Bibliothèques importées avec succès
📅 Date d'exécution: 2025-12-25 14:47:44
📋 Configuration chargée
   Serveur: boughida.com
   Base de données: techstore_erp
✅ Connexion MySQL réussie!
   Base de données active: techstore_erp
   Nombre de tables: 8

🚀 EXTRACTION DES TABLES MySQL

📊 Extraction: table_sales... ✅ 25000 lignes | 7 colonnes
📊 Extraction: table_products... ✅ 38 lignes | 5 colonnes
📊 Extraction: table_reviews... ✅ 3000 lignes | 5 colonnes
📊 Extraction: table_customers... ✅ 1200 lignes | 3 colonnes
📊 Extraction: table_stores... ✅ 12 lignes | 3 colonnes
📊 Extraction: table_cities... ✅ 12 lignes | 3 colonnes
📊 Extraction: table_categories... ✅ 5 lignes | 2 colonnes
📊 Extraction: table_subcategories... ✅ 15 lignes | 3 colonnes

✅ Extraction MySQL terminée!
📦 8 tables extraites

📋 APERÇU DES DONNÉES EXTRAITES


📊 Table: sales
   Dimensions: 25000 lignes × 7 colonnes
   Colonnes: Trans_ID, Date, Store_ID, Product_ID, Customer_ID...
   Aperçu:


,Trans_ID,Date,Store_ID,Product_ID,Customer_ID,Quantity,Total_Revenue
0,1,2024-07-02 17:15:00,3,P125,C0200,2,9000.0
1,2,2023-07-01 13:52:00,11,P136,C0883,1,2500.0
2,3,2025-01-05 18:56:00,7,P106,C0669,1,320000.0


------------------------------------------------------------

📊 Table: products
   Dimensions: 38 lignes × 5 colonnes
   Colonnes: Product_ID, Product_Name, SubCat_ID, Unit_Price, Unit_Cost...
   Aperçu:


,Product_ID,Product_Name,SubCat_ID,Unit_Price,Unit_Cost
0,P100,HP Victus 15,1,125000.0,87901.0
1,P101,Dell XPS 13,1,260000.0,167880.0
2,P102,MacBook Air M2,1,195000.0,141052.0


------------------------------------------------------------

📊 Table: reviews
   Dimensions: 3000 lignes × 5 colonnes
   Colonnes: Review_ID, Product_ID, Customer_ID, Rating, Review_Text...
   Aperçu:


...

✓ Nettoyage terminé → 0 lignes restantes

4. Imputation

5. Suppression des doublons
✓ 0 doublons supprimés → 0 lignes

6. Création profit_margin


,order_value_EUR,cost,profit_margin



7. Transactions en perte
Nombre de ventes en perte : 0

8. Standardisation colonnes texte

9. Colonnes dérivées + top catégories

10. Suppression incohérences
✓ 0 lignes incohérentes supprimées → 0 lignes valides
💾 Fichiers cleaned sauvegardés dans data/transformed/

11. Ajout 3 lignes NaN + backward fill
✓ 3 lignes ajoutées et backward fill appliqué

12. Scaling (MinMax & Standard)


,order_value_EUR_orig,cost_orig,profit_margin_orig,order_value_EUR_minmax,cost_minmax,profit_margin_minmax,order_value_EUR_std,cost_std,profit_margin_std
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



🎉 PARTIE 2 TERMINÉE AVEC SUCCÈS !
Toutes les transformations sont complètes et robustes.
